In [ ]:
"""
Created on Mon Jul 10 18:47 2023

Use cdo to regrid the variables to stereographic grid with cdo
to be used after "custom_lsmask_christoph.ipynb"

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Transformer
import pandas as pd
import sys,os
from cdo import Cdo
import time
from tqdm.notebook import tqdm

In [ ]:
%matplotlib qt5

In [ ]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

PREPARE PATH AND CONTEXT

In [ ]:
nemo_run = 'isfru94' # 'EPM026','EPM031', 'EPM034'
if nemo_run in ['ctrl94']:
    yy_start = 1982
    yy_end = 2013
elif nemo_run in ['isf94','isfru94']:
    yy_start = 2014
    yy_end = 2100

In [ ]:
outputpath='/bettik/burgardc/DATA/SUMMER_PAPER/interim/NEMO_'+nemo_run+'_ANT_STEREO/'

REGRIDDING THE MELT

In [ ]:
for yy in tqdm(range(yy_start,yy_end+1)):
    check = xr.open_dataset(outputpath+'variables_of_interest_'+str(yy)+'_Ant.nc')
    check['melt_cavity'] = check['fwfisf'].where(np.isfinite(check['fwfisf']),0)
    check2 = check[['melt_cavity']]
    check2.to_netcdf(outputpath+'cavity_melt_'+str(yy)+'_Ant.nc')

In [ ]:
for yy in tqdm(range(yy_start,yy_end+1)):
    #print('computing ',yy)
    time_start = time.time()
    cdo.remapbil(outputpath+'stereo_grid.nc', input = outputpath+'cavity_melt_'+str(yy)+'_Ant.nc', output = outputpath+'cavity_melt_'+str(yy)+'_Ant_stereo.nc')
    timelength = time.time() - time_start
    #print(timelength)

REGRIDDING ALL OTHER VARIABLES OF INTEREST

In [ ]:
for yy in tqdm(range(yy_start,yy_end+1)): #restart at 1998
#for yy in tqdm(range(2056,yy_end+1)): #
    print('computing ',yy)
    time_start = time.time()
    cdo.remapbil(outputpath+'stereo_grid.nc', input = outputpath+'variables_of_interest_'+str(yy)+'_Ant.nc', output = outputpath+'variables_of_interest_'+str(yy)+'_Ant_stereo.nc')
    timelength = time.time() - time_start
    print(timelength)

FINISHED? GOOD! THEN YOU ARE READY TO GO TO THE PRE-PROCESSING OF THE MASKS AND TEMPERATURE & SALINITY PROFILES =)